In [1]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import os
from sklearn import preprocessing
from torch import cuda
from torch.utils.data.dataloader import DataLoader

AttributeError: module 'torch.jit' has no attribute 'unused'

In [58]:
sys.path.append('../')
from mymodels.resnet_base_network import ResNet18
from mydata.imageloader import MyDataset, psnrDataUnit

In [59]:
batch_size = 64
data_transforms = torchvision.transforms.Compose([transforms.ToTensor()])

In [60]:
config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)

In [61]:
#generating index dict
import random
import glob

indexDict = dict()
file_path = "/home/hong/dir1/PyTorch-BYOL/writing2.txt"
# open index file
f = open(file_path, 'r')
lines = f.readlines()
for line in lines:
    listed = line.split(" ")  # split by spaces
    if listed[0] not in indexDict:
        # dict has img1_name
        indexDict[listed[0]] = list()
    tmp_unit = psnrDataUnit(listed[1], listed[2], listed[3])
    indexDict[listed[0]].append(tmp_unit)
f.close()
# check the loading is valid or not
indexDictKeys = list(indexDict.keys())
random.shuffle(indexDictKeys) 

In [62]:
image_path = '/mnt/URP_DS/HR' # without last slash
test_data_path = '/mnt/URP_DS/HR'  #without last slash

img1_idx = 0

train_img_psnr_list = []
for p_unit in indexDict[indexDictKeys[img1_idx]]:
    train_img_psnr_list.append((image_path + '/'+p_unit.getimg2()+'.png', p_unit.getonetwo()))
# train_image_paths= np.array(train_image_paths).flatten().tolist()
# random.shuffle(train_image_paths)  # ??? necessary??

print('train_image_path example: ', train_img_psnr_list[0], train_img_psnr_list[10])
print(len(train_img_psnr_list))

# split train valid from train paths (80,20)
# train_img_psnr_list, valid_img_psnr_list = train_img_psnr_list[:int(0.8*len(train_img_psnr_list))], train_img_psnr_list[int(0.8*len(train_img_psnr_list)):]

# create the test_image_paths
test_img_psnr_list = []
for p_unit in indexDict[indexDictKeys[img1_idx]]:
    test_img_psnr_list.append((image_path + '/'+p_unit.getimg2()+'.png', p_unit.getonetwo()))


train_image_path example:  ('/mnt/URP_DS/HR/44_22.png', 51.27391754) ('/mnt/URP_DS/HR/67_6.png', 44.79727264)
1613


In [63]:
# device = 'cpu'
device = 'cuda' if cuda.is_available() else 'cpu'
print(f"Training with: {device}")

train_dataset = MyDataset(train_img_psnr_list, transform=data_transforms)
# test_dataset = MyDataset(test_img_psnr_list, transform=data_transforms)
test_dataset = MyDataset(train_img_psnr_list, transform=data_transforms)
single_dataset = MyDataset([('/mnt/URP_DS/HR/'+indexDictKeys[img1_idx]+'.png', 40)], transform=data_transforms)  # 40-> meaningless

Training with: cpu


In [64]:
print("Input shape:", train_dataset[0][0].shape)

Input shape: torch.Size([3, 512, 512])


In [65]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True, pin_memory=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True, pin_memory=True)
                          
single_loader = DataLoader(single_dataset, batch_size=1,
                          num_workers=0, drop_last=False, shuffle=True, pin_memory=True)

In [66]:
encoder = ResNet18(**config['network'])
output_feature_dim = encoder.projetion.net[0].in_features
print(output_feature_dim)

512


In [67]:
#load pre-trained parameters
#load_params = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/Jul29_18-37-02_mango2/checkpoints/model.pth'),
#                         map_location=torch.device(torch.device(device)))
# epoch: 10
load_params = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/Sep26_15-10-29_mango2/checkpoints/model.pth'),
                        map_location=torch.device(torch.device(device)))

#load_params = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/Jul29_01-32-09_mango2/checkpoints/model.pth'),
#                         map_location=torch.device(torch.device(device)))\

# hong2
load_params2 = torch.load(os.path.join('/home/hong/dir1/PyTorch-BYOL/runs/using0928/checkpoints/model.pth'),
                        map_location=torch.device(torch.device(device)))

if 'online_network_state_dict' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameters successfully loaded.")

# remove the projection head
encoder = torch.nn.Sequential(*list(encoder.children())[:-1])    
encoder = encoder.to(device)

Parameters successfully loaded.


In [68]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

In [69]:
# Neural Network Class
class MyOne(torch.nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
        super().__init__()
        self.linear1 = torch.nn.Linear(D_in, H1)
        self.linear2 = torch.nn.Linear(H1, H2)
        self.linear3 = torch.nn.Linear(H2, H3)
        self.linear4 = torch.nn.Linear(H3, D_out)
    
    def forward(self, x):
        x = torch.nn.functional.relu(self.linear1(x))
        x = torch.nn.functional.relu(self.linear2(x))
        x = torch.nn.functional.relu(self.linear3(x))
        x = self.linear4(x)
        return x    

In [70]:
# logreg = LogisticRegression(output_feature_dim*2, 10)
# logreg = logreg.to(device)
mymo = MyOne(output_feature_dim*2, 512, 256, 64, 1)
mymo = mymo.to(device)
# 모델의 state_dict 출력
print("Model's state_dict:")
for param_tensor in mymo.state_dict():
    print(param_tensor, "\t", mymo.state_dict()[param_tensor].size())

Model's state_dict:
linear1.weight 	 torch.Size([512, 1024])
linear1.bias 	 torch.Size([512])
linear2.weight 	 torch.Size([256, 512])
linear2.bias 	 torch.Size([256])
linear3.weight 	 torch.Size([64, 256])
linear3.bias 	 torch.Size([64])
linear4.weight 	 torch.Size([1, 64])
linear4.bias 	 torch.Size([1])


In [71]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    for i, (x, y) in enumerate(loader):
        print(x.shape)
        with torch.no_grad():
            feature_vector = encoder(x)
            x_train.extend(feature_vector)
            y_train.extend(y.numpy())

            
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

In [72]:
encoder.eval()
x_train, y_train = get_features_from_encoder(encoder, train_loader)
x_test, y_test = get_features_from_encoder(encoder, test_loader)

if len(x_train.shape) > 2:
    print(x_train.shape)
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

torch.Size([64, 3, 512, 512])
torch.Size([64, 3, 512, 512])
torch.Size([64, 3, 512, 512])
torch.Size([64, 3, 512, 512])
torch.Size([64, 3, 512, 512])


In [ ]:
x_single, y_single = get_features_from_encoder(encoder, single_loader)
single_list = []
for i in range (0,x_train.shape[0]):
    single_list.append(x_single)
x_single1 = torch.stack(single_list, 1)
x_single1 = x_single1.squeeze()
print(x_single1.shape)

x_train = torch.cat((x_train, x_single1), 1)

single_list.clear()
for i in range (0,x_test.shape[0]):
    single_list.append(x_single)
x_single2 = torch.stack(single_list, 1)
x_single2 = x_single2.squeeze()
print(x_single2.shape)
x_test = torch.cat((x_test, x_single2), 1)

print(x_train.shape, x_test.shape)

torch.Size([1, 3, 512, 512])
torch.Size([1317, 512])
torch.Size([330, 512])
torch.Size([1317, 1024]) torch.Size([330, 1024])


In [ ]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):

    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=64, shuffle=False)
    return train_loader, test_loader

In [ ]:
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(np.float32)
x_test = scaler.transform(x_test).astype(np.float32)

In [ ]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)

In [ ]:
running_loss_history = []
running_logit_history = []

optimizer = torch.optim.Adam(mymo.parameters(), lr=3e-4)
# criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.L1Loss()
eval_every_n_epochs = 10

# device = 'cuda' if cuda.is_available() else 'cpu'
# print(f"Training with: {device}")

for epoch in range(200):
#     train_acc = []
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        logits = mymo(x)
        running_logit_history.append(1/logits[0].item())
        # predictions = torch.argmax(logits, dim=1)
        
        loss = criterion(logits, y)
        running_loss_history.append(loss)
        
        loss.backward()
        optimizer.step()
    
    # total = 0
    if epoch % eval_every_n_epochs == 0:
        # correct = 0
        validation_psnrsum_history = []
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            logits = mymo(x)
            validation_psnrsum_history.append(1/logits[0].item()-1/y[0].item()])
            # predictions = torch.argmax(logits, dim=1)
            
            # total += y.size(0)
            # correct += (predictions == y).sum().item()
        mymax = max(validation_psnrsum_history)
        mymin = min(validation_psnrsum_history)
        avg = sum(validation_psnrsum_history, 0.0)/len(validation_psnrsum_history)
        prediction = (mymax, mymin, avg)
        # acc = 100 * correct / total
        print("max: %f, min: %f, avg: %f" % prediction)
        print(validation_psnrsum_history)


/home/hong/anaconda3/envs/cuda11/lib/python3.8/site-packages/torch/nn/modules/loss.py:97: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/hong/anaconda3/envs/cuda11/lib/python3.8/site-packages/torch/nn/modules/loss.py:97: UserWarning: Using a target size (torch.Size([37])) that is different to the input size (torch.Size([37, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


max: 40.154814, min: 39.489699, avg: 39.859973
max: 40.358824, min: 40.099970, avg: 40.220761
max: 40.650260, min: 40.385801, avg: 40.546250
max: 40.802886, min: 40.683036, avg: 40.760205
max: 41.343215, min: 41.066036, avg: 41.267908
max: 40.445629, min: 40.176594, avg: 40.356597
max: 40.449566, min: 40.133184, avg: 40.349050
max: 40.056892, min: 39.548082, avg: 39.874135
max: 40.678843, min: 40.504375, avg: 40.588872
max: 40.225704, min: 40.111145, avg: 40.187753
max: 40.647847, min: 40.527827, avg: 40.605909
max: 40.040186, min: 39.807670, avg: 39.953034
max: 40.678313, min: 40.376155, avg: 40.579920
max: 40.065867, min: 40.025631, avg: 40.050479
max: 40.352265, min: 39.980831, avg: 40.199831
max: 39.974359, min: 39.761159, avg: 39.841047
max: 41.044185, min: 40.847398, avg: 40.979551
max: 40.918891, min: 40.877143, avg: 40.897500
max: 40.710467, min: 40.587911, avg: 40.628555
max: 39.676947, min: 39.456167, avg: 39.578801


In [ ]:
print(running_logit_history[-4:])
print(len(running_logit_history))
print(len(validation_psnrsum_history))

[40.98486508538505, 40.758923225888644, 40.6069691960277, 40.49976750410451]
4200
6
